# Exploring pure_callback

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

def f_host(x):
  # call a numpy (not jax.numpy) operation:
  print("f_host") # 因为是 pure_callback 所以不要使用这个
  return np.sin(x).astype(x.dtype)  # 返回一个新数组 每个元素的类型与x相同

def f(x):
  result_form = jax.ShapeDtypeStruct(x.shape, x.dtype)
  print("flag")
  #return jax.pure_callback(f_host, result_shape, x, vmap_method='sequential')
  # 会被转化成 numpy 数组喂进 f_host 结果出来之后会被再转化成 jax 数组
  print(jax.pure_callback(f_host, result_form, x, vmap_method='sequential'))
  # print(jax.pure_callback(f_host, None, x, vmap_method='sequential'))  # 如果写成 None 那么f_host里面的打印语句就不会执行

  print("f_host done")

x = jnp.arange(5.0) # 32位浮点数
f(x)

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

def f_host(x):
  # call a numpy (not jax.numpy) operation:
  return np.sin(x).astype(x.dtype)

def f(x):
  result_shape = jax.ShapeDtypeStruct(x.shape, x.dtype)
  return jax.pure_callback(f_host, result_shape, x, vmap_method='sequential')

x = jnp.arange(5.0)
f(x)

jax.jit(f)(x)

In [ ]:
def body_fun(_, x):
  return _, f(x)
jax.lax.scan(body_fun, None, jnp.arange(6.0))[1]

In [ ]:
jax.vmap(f)(x)

In [ ]:
# jax.grad(f)(x)

# ValueError: Pure callbacks do not support JVP. Please use `jax.custom_jvp` to use callbacks while taking gradients.

jax.pure_callback 参数总结

jax.pure_callback(callback, result_shape_dtypes, *args, vectorized=False, vmap_method='sequential', **kwargs)

callback:

你的 Python 函数: 在 CPU (宿主端) 执行，可以包含 NumPy 等非 JAX 操作。
输入 JAX 数组会转为 NumPy 数组，输出必须是 NumPy 数组。
result_shape_dtypes:

结果描述: 告诉 JAX 你的 callback 函数将返回什么形状 (shape) 和数据类型 (dtype) 的结果。
通常是 jax.ShapeDtypeStruct 对象或其 Pytree。JAX 在编译时需要这个静态信息。
*args:

输入参数: 传递给 callback 函数的位置参数。
vectorized (可选, 默认为 False):

向量化回调: 如果 True，表示你的 callback 函数本身就能处理批处理输入 (当 pure_callback 被 jax.vmap 时)。
vmap_method (可选, 默认为 'sequential'):

vmap 方式: 当 pure_callback 被 jax.vmap 且 vectorized=False 时，callback 如何处理批处理数据 (如：'sequential' 顺序执行，'parallel' 并行执行)。
**kwargs (可选):

关键字输入: 传递给 callback 函数的关键字参数。
jax.lax.scan 参数总结

jax.lax.scan(f, init, xs, length=None, reverse=False, unroll=1)

f (通常也叫 body_fun):

循环体函数: 在每个迭代步骤中执行的函数。
函数签名必须是：def body_fun(carry, x): -> (new_carry, y)
carry: 上一步的“携带状态”。
x: 来自输入序列 xs 的当前元素。
new_carry: 更新后的“携带状态”，传给下一步。
y: 当前步骤的输出，会被收集起来。
init:

初始状态: “携带状态 (carry)”的初始值。
xs:

输入序列: 要迭代处理的 JAX 数组或 Pytree。scan 会沿着它的主轴进行迭代。
length (可选):

迭代次数: 指定循环执行的固定次数，而不是从 xs 的长度推断。
reverse (可选, 默认为 False):

反向扫描: 如果为 True，则反向遍历输入序列 xs。
unroll (可选, 默认为 1):

循环展开: 编译器优化参数，指定循环展开的程度。
scan 返回值: (final_carry, ys_stacked)

final_carry: 最后一个 new_carry。
ys_stacked: 所有 y 输出堆叠成的数组。

In [ ]:
def print_something():
  print('printing something')
  return np.int32(0)

@jax.jit
def f1():
  return jax.pure_callback(print_something, np.int32(0)) # 省略了参数
f1();

In [ ]:
@jax.jit
def f2():
  jax.pure_callback(print_something, np.int32(0))
  return 1.0
f2(); # 这里的 print_something 不会被执行。实际上 callback 函数的返回值没有被用到，所以 callback 函数根本不会被执行

所以jax中pure_callback有什么用处？

总结来说，jax.pure_callback 的主要用处是：

当你需要在 JAX 的高性能、可转换的计算环境中，集成那些本质上不属于这个环境的、在宿主端执行的 Python/NumPy 代码时，它提供了一个必要的接口。它非常有用，但也因为其固有的开销和限制，应该在真正需要的时候才审慎使用。

# Exploring io_callback

In [ ]:
from jax.experimental import io_callback
from functools import partial

global_rng = np.random.default_rng(0)

def host_side_random_like(x):
  """Generate a random array like x using the global_rng state"""
  # We have two side-effects here:
  # - printing the shape and dtype
  # - calling global_rng, thus updating its state
  print(f'generating {x.dtype}{list(x.shape)}')
  return global_rng.uniform(size=x.shape).astype(x.dtype)

@jax.jit
def numpy_random_like(x):
  return io_callback(host_side_random_like, x, x)

x = jnp.zeros(5)
numpy_random_like(x)

In [ ]:
jax.vmap(numpy_random_like)(x)

In [ ]:
# 这里的 ordered=True 会导致每个 io_callback 都是顺序执行的
# 但是这和 vmap 有冲突
# 所以会报错

@jax.jit
def numpy_random_like_ordered(x):
  return io_callback(host_side_random_like, x, x, ordered=True)
# 这里的 ordered=True 会导致每个 io_callback 都是顺序执行的
# 但是这和 vmap 有冲突

jax.vmap(numpy_random_like_ordered)(x)

In [ ]:
# scan 和 io_callback 结合使用
# while_loop 也可以和 io_callback 结合使用

def body_fun(_, x):
  return _, numpy_random_like_ordered(x)
jax.lax.scan(body_fun, None, jnp.arange(5.0))[1]

In [ ]:
# Like pure_callback, io_callback fails under automatic differentiation if it is passed a differentiated variable:
# 如同 pure_callback，io_callback 在传入一个被微分的变量时会失败 因为callback 函数不是由 JAX 计算的

jax.grad(numpy_random_like)(x)

In [ ]:
# 当然啦，如果callback 函数和需要微分的变量没有关系 那么就没事了

@jax.jit
def f(x):
  io_callback(lambda: print('hello'), None)
  return x

jax.grad(f)(1.0);

# Exploring debug.callback

In [ ]:
from jax import debug

def log_value(x):
  # This could be an actual logging call; we'll use
  # print() for demonstration
  print("value:", x)

@jax.jit
def f(x):
  debug.callback(log_value, x)
  return x

f(1.0);

In [ ]:
x = jnp.arange(5.0)
jax.vmap(f)(x);

In [ ]:
jax.grad(f)(1.0);

# Example: pure_callback with custom_jvp

In [ ]:
import jax
import jax.numpy as jnp
import scipy.special

def jv(v, z):
  v, z = jnp.asarray(v), jnp.asarray(z)

  # Require the order v to be integer type: this simplifies
  # the JVP rule below.
  assert jnp.issubdtype(v.dtype, jnp.integer)

  # Promote the input to inexact (float/complex).
  # Note that jnp.result_type() accounts for the enable_x64 flag.
  z = z.astype(jnp.result_type(float, z.dtype))

  # Wrap scipy function to return the expected dtype.
  _scipy_jv = lambda v, z: scipy.special.jv(v, z).astype(z.dtype)

  # Define the expected shape & dtype of output.
  result_shape_dtype = jax.ShapeDtypeStruct(
      shape=jnp.broadcast_shapes(v.shape, z.shape),
      dtype=z.dtype)

  # Use vmap_method="broadcast_all" because scipy.special.jv handles broadcasted inputs.
  return jax.pure_callback(_scipy_jv, result_shape_dtype, v, z, vmap_method="broadcast_all")

In [ ]:
jv = jax.custom_jvp(jv)

@jv.defjvp
def jv_jvp(primals, tangents):
  v, z = primals
  _, z_dot = tangents  # Note: v_dot is always 0 because v is integer.
  jv_minus_1, jv_plus_1 = jv(v - 1, z), jv(v + 1, z)
  djv_dz = jnp.where(v == 0, -jv_plus_1, 0.5 * (jv_minus_1 - jv_plus_1))
  return jv(v, z), z_dot * djv_dz

In [ ]:
j1 = partial(jv, 1)
print(jax.grad(j1)(2.0))

In [ ]:
jax.hessian(j1)(2.0)